In [ ]:
import isaacgym

In [ ]:
from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed

In [ ]:
from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, get_rlgames_env_creator
from rl_games.common import env_configurations, vecenv
from rl_games.torch_runner import Runner
from rl_games.algos_torch import model_builder
import isaacgymenvs
from omegaconf import DictConfig, OmegaConf

In [ ]:
cfg = OmegaConf.load("cfg/config.yaml")
cfg.task_name="TrifingerNYU"
cfg.num_envs=1
cfg.task=OmegaConf.load("cfg/task/TrifingerNYU.yaml")

In [ ]:
def create_env_thunk(**kwargs):
        envs = isaacgymenvs.make(
            
            cfg.seed,
            cfg.task,
            cfg.num_envs,
            cfg.sim_device,
            cfg.rl_device,
            headless=True,
            cfg=cfg,
            **kwargs,
        )
        return envs

In [ ]:
envs = create_env_thunk()

In [ ]:
fid = [5, 12, 19]
jacobian_fingertip_linear = envs._jacobian[:, fid, :3, :]
jacobian_fingertip_linear = jacobian_fingertip_linear.view(
                envs.num_envs, 
                3 * envs._dims.NumFingers.value, 
                envs._dims.GeneralizedCoordinatesDim.value)

In [ ]:
# get fingertip states
fingertip_state = envs._rigid_body_state[:, envs._fingertip_indices]
fingertip_position = fingertip_state[:, :, 0:3].reshape(envs.num_envs, 9)
fingertip_velocity = fingertip_state[:, :, 7:10].reshape(envs.num_envs, 9)

q = envs._dof_position
dq = envs._dof_velocity

In [ ]:
import trifinger_simulation
import numpy as np
import pinocchio as pin

In [ ]:
platform = trifinger_simulation.TriFingerPlatform()
trifinger = platform.simfinger
rmodel = trifinger.kinematics.robot_model
rdata = trifinger.kinematics.data
ftip_id = trifinger.kinematics.tip_link_ids

In [ ]:
q_np = q[0].cpu().numpy()
dq_np = dq[0].cpu().numpy()

In [46]:
trifinger.kinematics.forward_kinematics(q_np)

[[0.09378661423891241, 0.05057204570721733, -0.02651110164976367],
 [-0.002945821003218333, -0.10625958173664683, -0.026151022087087444],
 [-0.09068988308828807, 0.05593559329825963, -0.026511101133743137]]

In [47]:
fingertip_position

tensor([[ 0.0938,  0.0506, -0.0265, -0.0029, -0.1063, -0.0262, -0.0907,  0.0559,
         -0.0265]], device='cuda:0')

In [ ]:
pin.computeJointJacobians(rmodel, rdata, q_np)
pin.framesForwardKinematics(rmodel, rdata, q_np)
J_ = []
for i in range(3):
    J_.append(pin.getFrameJacobian(
        rmodel,
        rdata,
        ftip_id[i],
        pin.ReferenceFrame.LOCAL_WORLD_ALIGNED,
    )[:3, :])
J = np.vstack(J_)

In [48]:
J @ dq_np # pinocchio, local world aligned

array([-0.10903541,  0.1157873 ,  0.39606528,  0.19653007,  0.06740448,
        0.52209394, -0.04569383, -0.15214085,  0.39611274])

In [39]:
jacobian_fingertip_linear[0] @ dq[0] # isaac

tensor([-0.1090,  0.1158,  0.3961,  0.1965,  0.0674,  0.5221, -0.0457, -0.1521,
         0.3961], device='cuda:0')

In [40]:
fingertip_velocity # isaac direct readings

tensor([[-0.0997,  0.1164, -0.0268,  0.1915,  0.0569, -0.0390, -0.0509, -0.1443,
         -0.0268]], device='cuda:0')